# 图像卷积
互相关运算

In [111]:
import torch
from torch import nn
from d2l import torch as d2l



In [112]:
# X:input 
# K:kennel
def corr2d(X,K):
    h,w = K.shape
    #Y:output
    Y = torch.zeros(X.shape[0] - h +1, X.shape[1]-w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return Y

In [113]:
X = torch.range(0,8,requires_grad=True).reshape(3,3)

K = torch.range(0,3).reshape(2,2)

Y = corr2d(X,K)
Y


C:\Users\Tamako\AppData\Local\Temp\ipykernel_15008\190142648.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  X = torch.range(0,8,requires_grad=True).reshape(3,3)
C:\Users\Tamako\AppData\Local\Temp\ipykernel_15008\190142648.py:3: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  K = torch.range(0,3).reshape(2,2)


tensor([[19., 25.],
        [37., 43.]], grad_fn=<CopySlices>)

实现二维卷积层

In [114]:
class Conv2D(nn.Module):
    def __init__(self,kernal_size):
        super().__init__()
        #参数初始化
        self.weight = nn.Parameter(torch.rand(kernal_size))
        #这里是初始化一列0，列数位置
        # 通过相加，自动确定
        self.bias = nn.parameter(torch.zeros(1))

    def forward(self,x):
        return corr2d(x,self.weight) + self.bias

In [115]:

torch.arange(0,5).reshape(5,1) + torch.zeros(1)

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.]])

卷积层的一个简单应用： 检测图像中不同颜色的边缘

In [116]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [117]:
K = torch.tensor([[1.0, -1.0]])
K.shape

torch.Size([1, 2])

输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘
检查从0-1 or 从 1-0

In [118]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

卷积核K只可以检测垂直边缘

In [119]:
#X.t()#转置
corr2d(X.t(), K)


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习由X生成Y的卷积核

In [120]:
#要去学习一个kernnel
# 跟据input:X 和 output:Y
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))



for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')


batch 2, loss 6.734
batch 4, loss 1.463
batch 6, loss 0.382
batch 8, loss 0.120
batch 10, loss 0.043


所学的卷积核的权重张量

In [121]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9676, -1.0083]])